## Problem and motivation

Whenever we crowd-source annotation of data, we inevitably run into a problem of annotators' fallibility. Having many annotators which disagree with another, how do we extract the true labels from aggregating their annotations?

The most straightforward solution is to, for each question (instance to be labeled), take the mode of the votes (the label that annotators give most often).

This, however, is not ideal and there are computational approaches that offer better results. The most widely known of those is the Dawid-Skene algorithm ([Dawid and Skene, 1979](https://www.jstor.org/stable/2346806); further: DS).

One downside of DS is its relative slowness. [Sinha, Rao, and Balasubramanian (2018)](http://sentic.net/wisdom2018sinha.pdf) propose a faster version of DS called Fast Dawid-Skene which is almost eight times faster than original DS. However, this speedup, comes with a cost of slightly lower accuracy. To balance speed and accuracy, a hybrid version of the algorithm (HDS) is implemented that starts like the normal DS but switches to FDS once the rate of change in the metric falls below a specified threshold.

The following code is to a large extent a Julia version of the [Python implementation of FDS by Sinha, Rao, and Balasubramanian](https://github.com/sukrutrao/Fast-Dawid-Skene).



## Expectation Maximization algorithm - high-level description

Here, FDS has been implemented as an [expectation-maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm).

EM algorithm is a general iterative method for estimating latent variables of statistical models.

Given some set of observations, we attempt to discover the parameters of the process that generated these observations. For example if our observations are #### together with labels assigned to them by Mechanical Turk workers, we want to discover the true labels, i.e. to which class a #### actually belongs to. To obtain that information, we need to compute intermediate information, such as *error rates of individual workers* (the probability of each worker giving a wrong label to any datapoint) and *class marginals* (the probability of any question to have a particular true label i.e. frequencies of true labels in the data).

EM algorithms achieve their goal by iterating over two steps, called the maximization (M) step and the expectation (E) step. In the M-step, we estimate the values of latent variables, given our current assignment of labels (we choose the values that *maximize* their likelihood given the labels, hence the name of the step). In the E-step, we re-compute the values #CHECKPOINT 

To showcase how the three versions of the algorithm work, we will go through the generic version of the algorithm step-by-step. We will note points where DS, FDS, and HDS behave differently.

## Data

First, we need to load the data. Here, we will use the RTE dataset ([download link](http://ir.ischool.utexas.edu/square/data.html)).

In [1]:
using DawidSkeneAlgorithms

dataset = load_rte()
typeof(dataset)

VotingDataset

The `VotingDataset` struct stores the crowd counts as a 3D `Array` (`x`) array of shape: `[number_of_questions x number_of_participants x number_of_classes]`. True labels are represented as a categorical vector of integers.

```julia
struct VotingDataset <: AbstractDataset
    name::String
    x::AbstractArray{<:Real,3} # [n_questions x n_annotators x n_options ]
    y::Vector{Int} # Categorical vector
end
```

In [5]:
using StatsBase: countmap

counts = dataset.x
counts_values = countmap(counts)

@show typeof(counts)
@show size(counts)
@show counts_values
flush(stdout)

typeof(counts) = Array{Float64, 3}
size(counts) = (800, 164, 2)
counts_values = Dict(0.0 => 254400, 1.0 => 8000)


So we have 800 questions (i.e. instances to be labeled), 164 participants who assigned a label to each question and two possible labels.

For example, the `j`-th participant concluded that the `i`-th person should be given label...

In [6]:
using Random

n_questions, n_participants = size(counts)
i = rand(1:n_questions)
j = rand(1:n_participants)
answer = counts[i, j, :]

@show i
@show j
@show answer

if answer == [1, 0]
    println("Label 1")
elseif answer == [0, 1]
    println("Label 2")
else 
    println("Didn't answer")
end

i = 566
j = 83
answer = [0.0, 0.0]
Didn't answer


## Algorithm

[Source code](../src/DawidSkeneAlgorithms/src/dawidskene.jl)

### Voting algorithms

We define three variations of the Dawid-Skene algorithm as `struct`s on which we will dispatch the methods constituting the main `em` method.

We also as well as majority voting algorithm to have a baseline to which we can compare their results.

First, abstract types.

```julia
# DawidSkeneAlgorithms.jl

# Abstract

abstract type AbstractAlgorithm end

# ...

#= We have another intermedaite abstract type 
    because there is one voting algorithm 
    that is not a Dawid-Skene variant, namely MajorityVoting =#

abstract type AbstractVotingAlgorithm <: AbstractAlgorithm end
const AVA = AbstractVotingAlgorithm

abstract type AbstractDawidSkene <: AbstractVotingAlgorithm end
const ADS = AbstractDawidSkene
```

Now, concrete type, which we will gather into a vector for convenience.

```julia
# dawidskene.jl

struct FastDawidSkene <: ADS end
const FDS = FastDawidSkene

struct DawidSkene <: ADS end
const DS = DawidSkene

struct HybridDawidSkene <: ADS end
const HDS = HybridDawidSkene
struct HDS_phase2 <: ADS end

struct MajorityVoting <: AVA end
const MV = MajorityVoting

const VOTING_ALGORITHMS = [FDS(), DS(), HDS(), MV()]
```

### Expectation-maximization algorithm for voting algorithms

```julia
# dawidskene.jl

function em(
    alg::ADS,
    counts::AbstractArray{<:Real,3};
    tol=0.0001,
    CM_tol=0.005,
    max_iter=100,
    verbose::Bool=false
)
# ...
```

### Initialization 

First, we initialize `class_assignments` and a few other local variables.

```julia
# ...
    # History of class assignments
    class_assignments_history = []

    # Matrix of estimates of true classes: [num_of_questions x num_of_responses]
    class_assignments = initialize_class_assignments(alg, counts)

    # Number of the current iteration
    n_iter = 0
    # Whether the algorithm has converged
    converged = false
    # Prior probabilities (frequencies) of classes
    old_class_marginals = nothing
    # The probability of participant `k` labeling response `l` for a question whose correct answer is `j` [num_of_participants x num_of_classes x num_of_classes]
    old_error_rates = nothing
    negloglik = nothing
```

(`initialize_class_assignments` is defined in [dawidskene_utilities.jl](../src/DawidSkeneAlgorithms/src/dawidskene_utilities.jl))

### The loop

We iterate over the loop until converence. At each iteration, we execute the M-step and the E-step and calculate log likelihood.

```julia
# ...
while !converged
        n_iter += 1
        class_marginals, error_rates = m_step(alg, counts, class_assignments)
        class_assignments = e_step(alg, counts, class_marginals, error_rates)
        negloglik = calculate_negloglikelihood(counts, class_marginals, error_rates)
        
        # ...
    end
# ...
```

(`calculate_negloglikelihood` is defined in [dawidskene_utilities.jl](../src/DawidSkeneAlgorithms/src/dawidskene_utilities.jl))

### M-step

In the M-step we use observations (`counts`) and current assignments (`class_assignments`) to calculate new estimates of `class_marginals` and annotators' `error_rates`.

Note that this step is the same for all versions of the algorithm. However, it requires the algorithm (`::AVA`) to be passed for multiple dispatch since different methods of `m_step` are used by [other versions of the EM algorithm](../src/DawidSkeneAlgorithms/src/mixturemodels.jl).

```julia
function m_step(
    ::AVA,
    counts::AbstractArray{<:Real,3},
    class_assignments::AbstractArray{<:Real,2}
)::Tuple{AbstractArray{<:Real,2},AbstractArray{<:Real,3}} # class_marginals, error_rates

    n_questions, n_annotators, n_options = size(counts)
    class_marginals = sum(class_assignments; dims=1) ./ n_questions
    error_rates = zeros(n_annotators, n_options, n_options)
    for a in 1:n_annotators
        for o1 in 1:n_options
            for o2 in 1:n_options
                error_rates[a, o1, o2] = class_assignments[:, o1]' * counts[:, a, o2]
            end
            sum_over_responses = sum(error_rates[a, o1, :])
            if sum_over_responses > 0
                error_rates[a, o1, :] = error_rates[a, o1, :] / sum_over_responses
            end
        end
    end
    return class_marginals, error_rates
end
```

### E-step

In the E-step, we again use the observations, along with results of the most recent M-step to compute new assignments.

Here, again, there is one method for all versions of the DS. However, it uses a helper function `_e_step_estimate_classes!` that has two different methods that dispatch on the algorithm type. Both return `nothing` and modify `class_assignments` or `final_class_assignments` (depending on the algorithm) in-place.

```julia
function e_step(
    alg::ADS,
    counts::AbstractArray{<:Real,3},
    class_marginals::AbstractArray{<:Real,2},
    error_rates::AbstractArray{<:Real,3}
)::AbstractArray{<:Real,2} # class_assignments or final_class_assignments

    n_questions, n_annotators, n_options = size(counts)
    class_assignments = zeros(n_questions, n_options)
    final_class_assignments = zeros(n_questions, n_options)
    for q in 1:n_questions
        for o in 1:n_options
            estimate = class_marginals[o] * prod(error_rates[:, o, :] .^ counts[q, :, :])
            class_assignments[q, o] = estimate
        end
        _e_step_estimate_classes!(alg, q, class_assignments, final_class_assignments)
    end

    if typeof(alg) ∈ [DS, HDS]
        return class_assignments
    else # FDS / HDS_phase2
        return final_class_assignments
    end
end
```

The first method dispatches on `DawidSkene` and `HybridDawidSkene`.

```julia
function _e_step_estimate_classes!(
    ::Union{DS,HDS},
    i::Int,
    class_assignments::AbstractArray{<:Real,2},
    final_class_assignments::AbstractArray{<:Real,2}
)
    class_assignments_sum = sum(class_assignments[i, :])
    if class_assignments_sum > 0
        class_assignments[i, :] = class_assignments[i, :] / class_assignments_sum
    end
    return
end
```

You may be wondering

> If `_e_step_estimate_classes!` dispatches the same method for `DS` and `HDS` and no other function dispatches on the algorithm, then `DS` and `HDS` should work the same way...

The thing is, the other method dispatches on `FDS` **and** `HDS_phase2`

```julia
function _e_step_estimate_classes!(
    ::Union{FDS,HDS_phase2},
    i::Int,
    class_assignments::AbstractArray{<:Real,2},
    final_class_assignments::AbstractArray{<:Real,2}
)
    maxval = maximum(class_assignments[i, :])
    maxinds = argwhere(class_assignments[i, :], ==(maxval))
    final_class_assignments[i, sample(maxinds, 1)[1]] = 1
    return
end
```


And we change `alg` from `HDS` to `HDS_phase2` after the rate of change of `class_marginals` slows down below the threshold value of `CM_tol` which we passed as an argument to `em`.

Another argument `tol` specifies the rate of change of class marginals below which we decide that the algorithm has converged. The convergence can also occur if we have exceeded a specified maximum number of iterations.

```julia
# ...
    while !converged 
        # ...
    
        # Check for convergence
        if old_class_marginals ≠ nothing
            # How much have `class_marginals` and `error_rates` changed since last iteration
            
            class_marginals_diff = sum(abs.(class_marginals - old_class_marginals))
            error_rates_diff = sum(abs.(error_rates - old_error_rates))
            # If class marginals change rate goes below a certain threshold or we have done too many iterations consider the algorithm converged
            if class_marginals_diff < tol || n_iter >= max_iter
                converged = true
            # If using Hybrid Dawid-Skene and class marginals change rate goes below the threshold, switch the algorithm to phase 2, in which it works like Fast Dawid-Skene
            elseif alg isa HDS && class_marginals_diff ≤ CM_tol
                alg = HDS_phase2()
            end
        end

        old_class_marginals = class_marginals
        old_error_rates = error_rates

        if verbose
            @show n_iter
            @show negloglik
        end

        # Update history
        push!(class_assignments_history, class_assignments)
    end
# ...
```

So in short, Hybrid Dawid-Skene works just like normal Dawid-Skene *until* the rate of change of estimated class marginals decreases below a certain threshold. From that point onward, it works like Fast Dawid-Skene.

### Result

After convergence, we have the array `class_assignments` (`[num_of_questions x num_of_classes]`) which stores the probability distribution over the classes for each question.

We take the most probable class for each question and return it along with the log likelihood which was calculated after each pair of M- and E-steps.

```julia
    verbose && @show negloglik
    result = map(
        x -> x[2],
        argmax(class_assignments, dims=2)[:])
    return result, negloglik
end
```

## Run

Let's load everything and run the algorithms on the dataset

In [7]:
using BenchmarkTools
VOTING_ALGORITHMS

4-element Vector{DawidSkeneAlgorithms.AbstractVotingAlgorithm}:
 FastDawidSkene
 DawidSkene
 HybridDawidSkene
 MajorityVoting

In [8]:
alg_results = Dict{AVA,Any}()
alg_neglogliks = Dict{AVA,Float64}()

Dict{DawidSkeneAlgorithms.AbstractVotingAlgorithm, Float64}()

### Fast Dawid-Skene

In [9]:
result, negloglikelihood = @btime em($FDS(), $dataset.x) seconds=5
alg_results[FDS()] = result
alg_neglogliks[FDS()] = negloglikelihood
;

  36.023 ms (71644 allocations: 135.61 MiB)


In [10]:
println("Fast Dawid-Skene
Negative log-likelihood: $(round(negloglikelihood; digits=2))")

Fast Dawid-Skene
Negative log-likelihood: 3731.83


### Dawid-Skene

In [11]:
result, negloglikelihood = @btime em($DS(), $dataset.x) seconds=5
alg_results[DS()] = result
alg_neglogliks[DS()] = negloglikelihood
;

  93.047 ms (159787 allocations: 370.52 MiB)


In [12]:
println("Dawid-Skene
Negative log-likelihood: $(round(negloglikelihood; digits=2))")

Dawid-Skene
Negative log-likelihood: 3679.63


### Hybrid Dawid-Skene

In [13]:
result, negloglikelihood = @btime em($HDS(), $dataset.x) seconds=5
alg_results[HDS()] = result
alg_neglogliks[HDS()] = negloglikelihood
;

  78.285 ms (138375 allocations: 303.63 MiB)


In [14]:
println("Hybrid Dawid-Skene
Negative log-likelihood: $(round(negloglikelihood; digits=2))")

Hybrid Dawid-Skene
Negative log-likelihood: 3680.32


### Majority Voting

In [15]:
result, negloglikelihood = @btime em($MV(), $dataset.x) seconds=5
alg_results[MV()] = result
alg_neglogliks[MV()] = negloglikelihood
;

  4.216 ms (11920 allocations: 21.23 MiB)


In [16]:
println("Majority Voting
Negative log-likelihood: $(round(negloglikelihood; digits=2))")

Majority Voting
Negative log-likelihood: 3792.05


# Evaluation

As we can see, Hybrid Dawid-Skene is a little bit closer to Fast Dawid-Skene in speed but achieves log-likelihood comparable to normal Dawid-Skene.

What's important, these values are very close to those reported in [the paper](http://sentic.net/wisdom2018sinha.pdf) for the RTE dataset (Table 2) which strongly suggests that our implementation works as intended.

![](../images/FDS%20Paper%20-%20Table2.png)

We can also compare these results to the ground truth. For this dataset, they are represented by a Boolean vector. Results of the algorithms are represented as categorical vectors (here, binary `1`s and `2`s). We can evaluate how each algorithm performed by computing [mutual information](https://en.wikipedia.org/wiki/Mutual_information) between their predictions and the ground truth using a function from the [Clustering.jl](https://juliastats.org/Clustering.jl/stable/) package.

In [17]:
true_labels = dataset.y
println("Length: $(length(true_labels)); Counts: $(countmap(true_labels))")

Length: 800; Counts: Dict(0 => 400, 1 => 400)


In [18]:
using Clustering: mutualinfo

for (alg, results) in alg_results
    mi = round(mutualinfo(results, true_labels); digits=3)
    println("$alg: $mi\n")
end

FastDawidSkene: 0.609

MajorityVoting: 0.548

DawidSkene: 0.628

HybridDawidSkene: 0.628



Again, we see that `HDS` and `DS` achieve basically the same scores, with `FDS` lagging slightly behind.

The `em_avg` wrapper in [em_wrappers.jl](../src/DawidSkeneAlgorithms/src/em_wrappers.jl) runs a given algorithm on a given dataset a specified number of times and returns average negative log-likelihood scores, mutual information scores, or both, depending on the `return_type` parameter.